<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher/blob/main/mediapipe_object_detector_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project: /mediapipe/_project.yaml
Book: /mediapipe/_book.yaml

<link rel="stylesheet" href="/mediapipe/site.css">

# Object detection model customization guide

<table align="left" class="buttons">
  <td>
    <a href="https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/customization/object_detector.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/colab-logo-32px_1920.png" alt="Colab logo"> Run in Colab
    </a>
  </td>

  <td>
    <a href="https://github.com/googlesamples/mediapipe/blob/main/examples/customization/object_detector.ipynb" target="_blank">
      <img src="https://developers.google.com/static/mediapipe/solutions/customization/github-logo-32px_1920.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

MediaPipe オブジェクト検出ソリューションには、アプリケーションの機械学習 (ML) にすぐに使用できるモデルがいくつか用意されています。 ただし、提供されたモデルでカバーされていないオブジェクトを検出する必要がある場合は、独自のデータと MediaPipe Model Maker を使用して提供されたモデルのいずれかをカスタマイズできます。 このモデル変更ツールは、提供されたデータを使用してモデルを再構築します。 この方法は、新しいモデルをトレーニングするよりも速く、特定のアプリケーションにとってより役立つモデルを生成できます。

次のセクションでは、Model Maker を使用して、独自のデータを使用してオブジェクト検出用の事前構築モデルを再トレーニングし、その後 MediaPipe Object Detector で使用できるようにする方法を示します。 この例では、画像内の Android フィギュアを検出するために汎用オブジェクト検出モデルを再トレーニングします。

The MediaPipe object detection solution provides several models you can use immediately for machine learning (ML) in your application. However, if you need to detect objects not covered by the provided models, you can customize any of the provided models with your own data and MediaPipe Model Maker. This model modification tool rebuilds the model using data you provide. This method is faster than training a new model and can produce a model that is more useful for your specific application.

The following sections show you how to use Model Maker to retrain a pre-built model for object detection with your own data, which you can then use with the MediaPipe [Object Detector](https://developers.google.com/mediapipe/solutions/vision/object_detector). The example retrains a general purpose object detection model to detect android figurines in images.

## 設定

このセクションでは、モデルを再トレーニングするために開発環境をセットアップするための主要な手順について説明します。 これらの手順では、[Google Colab](https://colab.research.google.com/) を使用してモデルを更新する方法について説明します。また、独自の開発環境で Python を使用することもできます。 MediaPipe を使用するための開発環境のセットアップに関する一般情報 (プラットフォームのバージョン要件など) については、[Python のセットアップ ガイド](https://developers.google.com/mediapipe/solutions/setup_python) を参照してください。

**注意:** この MediaPipe ソリューション プレビューは初期リリースです。 [詳細](https://developers.google.com/mediapipe/solutions/about)。

モデルをカスタマイズするためのライブラリをインストールするには、次のコマンドを実行します。

## Setup

This section describes key steps for setting up your development environment to retrain a model. These instructions describe how to update a model using [Google Colab](https://colab.research.google.com/), and you can also use Python in your own development environment. For general information on setting up your development environment for using MediaPipe, including platform version requirements, see the [Setup guide for Python](https://developers.google.com/mediapipe/solutions/setup_python).

**Attention:** This MediaPipe Solutions Preview is an early release. [Learn more](https://developers.google.com/mediapipe/solutions/about).

To install the libraries for customizing a model, run the following commands:

In [ ]:
!python --version
!pip install --upgrade pip
!pip install mediapipe-model-maker

次のコードを使用して、必要な Python クラスをインポートします。

Use the following code to import the required Python classes:

In [ ]:
from google.colab import files
import os
import json
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import object_detector

## データの準備

オブジェクト検出用にモデルを再トレーニングするには、完成したモデルで識別できるようにするアイテムまたはクラスを含むデータセットが必要です。 これを行うには、パブリック データセットをトリミングしてユースケースに関連するクラスのみにするか、独自のデータセットをコンパイルするか、またはその両方を組み合わせます。データセットは、新しいモデルのトレーニングに必要なデータセットよりも大幅に小さくなる可能性があります。 たとえば、多くの参照モデルのトレーニングに使用される COCO データセットには、91 クラスのオブジェクトを含む数十万の画像が含まれています。 Model Maker を使用した転移学習は、推論精度の目標に応じて、より小さなデータセットを使用して既存のモデルを再トレーニングしても、良好なパフォーマンスを維持できます。 これらの手順では、2 種類のアンドロイド フィギュア、つまり 2 つのクラス、合計 62 個のトレーニング画像を含む小さなデータセットを使用します。

サンプル データセットをダウンロードするには、次のコードを使用します。

Retraining a model for object detection requires a dataset that includes the items, or classes, that you want the completed model to be able to identify. You can do this by trimming down a public dataset to only the classes that are relevant to your usecase, compiling your own dataset, or some combination of both, The dataset can be significantly smaller than what would be required to train a new model. For example, the [COCO](https://cocodataset.org/) dataset used to train many reference models contains hundreds of thousands of images with 91 classes of objects. Transfer learning with Model Maker can retrain an existing model with a smaller dataset and still perform well, depending on your inference accuracy goals. These instructions use a smaller dataset containing 2 types of android figurines, or 2 classes, with 62 total training images.

To download the example dataset, use the following code:

In [ ]:
!wget https://storage.googleapis.com/mediapipe-tasks/object_detector/android_figurine.zip
!unzip android_figurine.zip
train_dataset_path = "android_figurine/train"
validation_dataset_path = "android_figurine/validation"

This code stores the dataset at the directory location `android_figurine`. The directory contains two subdirectories for the training and validation datasets, located in `android_figurine/train` and `android_figurine/validation` respectively. Each of the train and validation datasets follow the COCO Dataset format described below.

### サポートされているデータセット形式
Model Maker Object Detection API は、次のデータセット形式の読み取りをサポートしています。

#### COCO 形式

COCO データセット形式には、すべての画像を保存する「data」ディレクトリと、すべての画像のオブジェクト注釈を含む単一の「labels.json」ファイルがあります。

### Supported dataset formats
Model Maker Object Detection API supports reading the following dataset formats:

#### COCO format

The COCO dataset format has a `data` directory which stores all of the images and a single `labels.json` file which contains the object annotations for all images.
```
<dataset_dir>/
  data/
    <img0>.<jpg/jpeg>
    <img1>.<jpg/jpeg>
    ...
  labels.json
```
where `labels.json` is formatted as:
```
{
  "categories":[
    {"id":1, "name":<cat1_name>},
    ...
  ],
  "images":[
    {"id":0, "file_name":"<img0>.<jpg/jpeg>"},
    ...
  ],
  "annotations":[
    {"id":0, "image_id":0, "category_id":1, "bbox":[x-top left, y-top left, width, height]},
    ...
  ]
}
```

#### PASCAL VOC format

The PASCAL VOC dataset format also has a `data` directory which stores all of the images, however the annotations are split up per image into corresponding xml files in the `Annotations` directory.
```
<dataset_dir>/
  data/
    <file0>.<jpg/jpeg>
    ...
  Annotations/
    <file0>.xml
    ...
```
where the xml files are formatted as:
```
<annotation>
  <filename>file0.jpg</filename>
  <object>
    <name>kangaroo</name>
    <bndbox>
      <xmin>233</xmin>
      <ymin>89</ymin>
      <xmax>386</xmax>
      <ymax>262</ymax>
    </bndbox>
  </object>
  <object>
    ...
  </object>
  ...
</annotation>
```

### データセットを確認する

「labels.json」ファイルからカテゴリを出力して、データセットの内容を確認します。 カテゴリは合計 3 つあるはずです。 インデックス 0 は常に、データセット内で使用されない可能性がある「バックグラウンド」クラスに設定されます。 `android` と `pig_android` という 2 つの非バックグラウンド カテゴリがあるはずです。

Verify the dataset content by printing the categories from the `labels.json` file. There should be 3 total categories. Index 0 is always set to be the `background` class which may be unused in the dataset. There should be two non-background categories of `android` and `pig_android`.

In [ ]:
with open(os.path.join(train_dataset_path, "labels.json"), "r") as f:
  labels_json = json.load(f)
for category_item in labels_json["categories"]:
  print(f"{category_item['id']}: {category_item['name']}")

データセットをより深く理解するには、いくつかのサンプル画像をその境界ボックスとともにプロットします。

To better understand the dataset, plot a couple of example images along with their bounding boxes.

In [ ]:
#@title トレーニングデータの視覚化
import matplotlib.pyplot as plt
from matplotlib import patches, text, patheffects
from collections import defaultdict
import math

def draw_outline(obj):
  obj.set_path_effects([patheffects.Stroke(linewidth=4,  foreground='black'), patheffects.Normal()])
def draw_box(ax, bb):
  patch = ax.add_patch(patches.Rectangle((bb[0],bb[1]), bb[2], bb[3], fill=False, edgecolor='red', lw=2))
  draw_outline(patch)
def draw_text(ax, bb, txt, disp):
  text = ax.text(bb[0],(bb[1]-disp),txt,verticalalignment='top'
  ,color='white',fontsize=10,weight='bold')
  draw_outline(text)
def draw_bbox(ax, annotations_list, id_to_label, image_shape):
  for annotation in annotations_list:
    cat_id = annotation["category_id"]
    bbox = annotation["bbox"]
    draw_box(ax, bbox)
    draw_text(ax, bbox, id_to_label[cat_id], image_shape[0] * 0.05)
def visualize(dataset_folder, max_examples=None):
  with open(os.path.join(dataset_folder, "labels.json"), "r") as f:
    labels_json = json.load(f)
  images = labels_json["images"]
  cat_id_to_label = {item["id"]:item["name"] for item in labels_json["categories"]}
  image_annots = defaultdict(list)
  for annotation_obj in labels_json["annotations"]:
    image_id = annotation_obj["image_id"]
    image_annots[image_id].append(annotation_obj)

  if max_examples is None:
    max_examples = len(image_annots.items())
  n_rows = math.ceil(max_examples / 3)
  fig, axs = plt.subplots(n_rows, 3, figsize=(24, n_rows*8)) # 3 columns(2nd index), 8x8 for each image
  for ind, (image_id, annotations_list) in enumerate(list(image_annots.items())[:max_examples]):
    ax = axs[ind//3, ind%3]
    img = plt.imread(os.path.join(dataset_folder, "images", images[image_id]["file_name"]))
    ax.imshow(img)
    draw_bbox(ax, annotations_list, cat_id_to_label, img.shape)
  plt.show()

visualize(train_dataset_path, 9)

### データセットを作成する

Dataset クラスには、COCO または PASCAL VOC データセットをロードするための 2 つのメソッドがあります。
* `Dataset.from_coco_folder`
* `Dataset.from_pascal_voc_folder`

android_figurines データセットは COCO データセット形式であるため、`from_coco_folder` メソッドを使用して、`train_dataset_path` と `validation_dataset_path` にあるデータセットを読み込みます。 データセットをロードすると、データは指定されたパスから解析され、標準化された [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord) 形式に変換され、後で使用するためにキャッシュされます。 同じデータセットの複数のキャッシュを保存しないようにするには、`cache_dir` の場所を作成し、すべてのトレーニングでそれを再利用する必要があります。

### Create dataset

The Dataset class has two methods for loading in COCO or PASCAL VOC datasets:
* `Dataset.from_coco_folder`
* `Dataset.from_pascal_voc_folder`

Since the android_figurines dataset is in the COCO dataset format, use the `from_coco_folder` method to load the dataset located at `train_dataset_path` and `validation_dataset_path`. When loading the dataset, the data will be parsed from the provided path and converted into a standardized [TFRecord](https://www.tensorflow.org/tutorials/load_data/tfrecord) format which is cached for later use. You should create a `cache_dir` location and reuse it for all your training to avoid saving multiple caches of the same dataset.

In [ ]:
train_data = object_detector.Dataset.from_coco_folder(train_dataset_path, cache_dir="/tmp/od_data/train")
validation_data = object_detector.Dataset.from_coco_folder(validation_dataset_path, cache_dir="/tmp/od_data/validation")
print("train_data size: ", train_data.size)
print("validation_data size: ", validation_data.size)

## 再学習過程

データの準備が完了したら、トレーニングデータで定義した新たな物体やクラスを認識するように再学習過程を開始することができる。以下の手順では、前のセクションで準備したデータを使用して画像分類モデルを再トレーニングし、2種類のアンドロイド フィギュアを認識する。

### Set retraining options

再トレーニングを実行するには、トレーニング データセットのほかに、モデルの出力ディレクトリとモデル アーキテクチャといういくつかの必要な設定があります。 HParams を使用して、出力ディレクトリの export_dir パラメータを指定します。 SupportedModels クラスを使用してモデル アーキテクチャを指定します。 物体検出ソリューションは、次のモデル アーキテクチャをサポートします。
* `MobileNet-V2`
* `MobileNet-MultiHW-AVG`
トレーニング パラメーターのより高度なカスタマイズについては、以下の [ハイパーパラメーター](#hyperparameters) セクションをご覧ください。

必要なパラメータを設定するには、次のコードを使用します。

In [ ]:
spec = object_detector.SupportedModels.MOBILENET_MULTI_AVG
hparams = object_detector.HParams(export_dir='exported_model')
options = object_detector.ObjectDetectorOptions(
    supported_model=spec,
    hparams=hparams
)

### 再学習過程


In [ ]:
model = object_detector.ObjectDetector.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options)

### モデルのパフォーマンスを評価する

モデルをトレーニングした後、検証データセットを用いてモデルを評価し、損失率と coco_metrics を出力します。 モデルのパフォーマンスを評価するための最も重要な指標は、通常、平均精度の「AP」ココメトリックです。

In [ ]:
loss, coco_metrics = model.evaluate(validation_data, batch_size=4)
print(f"Validation loss: {loss}")
print(f"Validation coco metrics: {coco_metrics}")

## モデルのエクスポート

モデルを作成した後、後でデバイス上のアプリケーションで使用できるように、モデルを Tensorflow Lite モデル形式に変換してエクスポートします。 エクスポートには、ラベルマップを含むモデルメタデータも含まれます。

In [ ]:
model.export_model()
!ls exported_model
files.download('exported_model/model.tflite')

## モデルの量子化

モデルを作成した後、後でデバイス上のアプリケーションで使用できるように、モデルを Tensorflow Lite モデル形式に変換してエクスポートします。 エクスポートにはモデルも含まれます。 モデル量子化は、比較的わずかな精度の低下のみでモデル サイズを削減し、予測速度を向上させることができるモデル変更手法です。

ガイドのこのセクションでは、モデルに量子化を適用する方法について説明します。 Model Maker は、オブジェクト検出器の 2 つの形式の量子化をサポートしています。
1. 量子化対応トレーニング: CPU 使用のための 8 ビット整数精度
2. トレーニング後の量子化: GPU使用のための16 ビット浮動小数点精度。

Model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy.

This section of the guide explains how to apply quantization to your model. Model Maker supports two forms of quantization for object detector:
1. Quantization Aware Training: 8 bit integer precision for CPU usage
2. Post-Training Quantization: 16 bit floating point precision for GPU usage

### 量子化を意識したトレーニング (int8 量子化)
量子化対応トレーニング (QAT) は、モデルを完全にトレーニングした後に行われる微調整ステップです。 この手法では、8 ビット整数量子化の精度の低さを考慮して、推論時間量子化をエミュレートするモデルをさらに調整します。 標準 CPU を備えたオンデバイス アプリケーションの場合は、Int8 精度を使用します。 詳細については、[TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/training) ドキュメントを参照してください。

量子化対応トレーニングを適用して int8 モデルにエクスポートするには、`QATHParams` 設定を作成し、`quantization_aware_training` メソッドを実行します。 `QATHParams` の詳細な使用法については、以下の **ハイパーパラメータ** セクションを参照してください。

### Quantization aware training (int8 quantization)
Quantization aware training (QAT) is a fine-tuning step which happens after fully training your model. This technique further tunes a model which emulates inference time quantization in order to account for the lower precision of 8 bit integer quantization. For on-device applications with a standard CPU, use Int8 precision. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/training) documentation.

To apply quantization aware training and export to an int8 model, create a `QATHParams` configuration and run the `quantization_aware_training` method. See the **Hyperparameters** section below on detailed usage of `QATHParams`.

In [ ]:
qat_hparams = object_detector.QATHParams(learning_rate=0.3, batch_size=4, epochs=10, decay_steps=6, decay_rate=0.96)
model.quantization_aware_training(train_data, validation_data, qat_hparams=qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

QAT ステップでは、トレーニングのパラメーターを調整するために複数の実行が必要になることがよくあります。 `create` メソッドを使用してモデル トレーニングを再実行する必要がないようにするには、QAT を再度実行するために、`restore_float_ckpt` メソッドを使用してモデルの状態を完全にトレーニングされた浮動小数点モデルに戻します (`create` メソッドの実行後)。

The QAT step often requires multiple runs to tune the parameters of training. To avoid having to rerun model training using the `create` method, use the `restore_float_ckpt` method to restore the model state back to the fully trained float model(After running the `create` method) in order to run QAT again.

In [ ]:
new_qat_hparams = object_detector.QATHParams(learning_rate=0.9, batch_size=4, epochs=15, decay_steps=5, decay_rate=0.96)
model.restore_float_ckpt()
model.quantization_aware_training(train_data, validation_data, qat_hparams=new_qat_hparams)
qat_loss, qat_coco_metrics = model.evaluate(validation_data)
print(f"QAT validation loss: {qat_loss}")
print(f"QAT validation coco metrics: {qat_coco_metrics}")

最後に、export_model を使用して int8 量子化モデルにエクスポートします。 export_model 関数は、quantization_aware_training が実行されたかどうかに応じて、float32 モデルまたは int8 モデルのいずれかに自動的にエクスポートします。

Finally, us the `export_model` to export to an int8 quantized model. The `export_model` function will automatically export to either float32 or int8 model depending on whether `quantization_aware_training` was run.

In [ ]:
model.export_model('model_int8_qat.tflite')
!ls -lh exported_model
files.download('exported_model/model_int8_qat.tflite')

### トレーニング後の量子化 (fp16 量子化)

トレーニング後のモデルの量子化は、比較的わずかな精度の低下のみでモデルのサイズを削減し、予測速度を向上させることができるモデル変更手法です。 このアプローチでは、たとえば 32 ビット浮動小数点数を 16 ビット浮動小数点数に変換することにより、モデルによって処理されるデータのサイズが削減されます。 GPU の使用には Float16 量子化が推奨されます。 詳細については、[TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/post_training) ドキュメントを参照してください。

まず、MediaPipe Model Maker 量子化モジュールをインポートします。

### Post-training quantization (fp16 quantization)

Post-training model quantization is a model modification technique that can reduce the model size and improve the speed of predictions with only a relatively minor decrease in accuracy. This approach reduces the size of the data processed by the model, for example by transforming 32-bit floating point numbers to 16-bit floats. Float16 quantization is reccomended for GPU usage. For more information, see the [TensorFlow Lite](https://www.tensorflow.org/model_optimization/guide/quantization/post_training) documentation.

First, import the MediaPipe Model Maker quantization module:

In [ ]:
from mediapipe_model_maker import quantization

for_float16() クラス メソッドを使用して QuantizationConfig オブジェクトを定義します。 この構成は、32 ビット浮動小数点数の代わりに 16 ビット浮動小数点数を使用するようにトレーニングされたモデルを変更します。 QuantizationConfig クラスの追加パラメーターを設定することで、量子化プロセスをさらにカスタマイズできます。

Define a QuantizationConfig object using the `for_float16()` class method. This configuration modifies a trained model to use 16-bit floating point numbers instead of 32-bit floating point numbers. You can further customize the quantization process by setting additional parameters for the QuantizationConfig class.

In [ ]:
quantization_config = quantization.QuantizationConfig.for_float16()

追加の quantization_config オブジェクトを使用してモデルをエクスポートし、トレーニング後の量子化を適用します。 以前に quantization_aware_training を実行したことがある場合は、まず、restore_float_ckpt を使用してモデルを float モデルに変換し直す必要があることに注意してください。

Export the model using the additional quantization_config object to apply post-training quantization. Note that if you previously ran `quantization_aware_training`, you must first convert the model back to a float model by using `restore_float_ckpt`.

In [ ]:
model.restore_float_ckpt()
model.export_model(model_name="model_fp16.tflite", quantization_config=quantization_config)
!ls -lh exported_model
files.download('exported_model/model_fp16.tflite')

##ハイパーパラメータ
ObjectDetectorOptions クラスを使用してモデルをさらにカスタマイズできます。このクラスには、SupportedModels、ModelOptions、および HParams の 3 つのパラメーターがあります。

SupportedModels enum クラスを使用して、トレーニングに使用するモデル アーキテクチャを指定します。 次のモデル アーキテクチャがサポートされています。

モバイルネット_V2
モバイルネット_V2_I320
MOBILENET_MULTI_AVG
MOBILENET_MULTI_AVG_I384
HParams クラスを使用して、モデルのトレーニングと保存に関連する他のパラメーターをカスタマイズします。

learning_rate: 勾配降下トレーニングに使用する学習率。 デフォルトは 0.3 です。
batch_size: トレーニングのバッチ サイズ。 デフォルトは 8 です。
エポック: データセットに対するトレーニング反復の数。 デフォルトは 30 です。
cosine_decay_epochs: コサイン減衰学習率のエポック数。 詳細については、「tf.keras.optimizers.schedules.CosineDecay」を参照してください。 デフォルトは None で、これはエポックに設定するのと同じです。
cosine_decay_alpha: コサイン減衰学習率のアルファ値。 詳細については、「tf.keras.optimizers.schedules.CosineDecay」を参照してください。 デフォルトは 1.0 で、コサイン減衰がないことを意味します。
ModelOptions クラスを使用して、モデル自体に関連するパラメーターをカスタマイズします。

l2_weight_decay: tf.keras.regulators.L2 で使用される L2 正則化ペナルティ。 デフォルトは 3e-5 です。
QATHParams クラスを使用して、量子化対応トレーニングのトレーニング パラメーターをカスタマイズします。

learning_rate: 勾配降下 QAT に使用する学習率。 デフォルトは 0.3 です。
batch_size: QAT のバッチ サイズ。 デフォルトは 8
エポック: データセットに対するトレーニング反復の数。 デフォルトは 15 です。
decay_steps: 指数関数的減衰の学習率減衰ステップ。 詳細については、「tf.keras.optimizers.schedules.ExponentialDecay」を参照してください。 デフォルトは 8
decay_rate: 指数関数的減衰の学習率減衰率。 詳細については、「tf.keras.optimizers.schedules.ExponentialDecay」を参照してください。 デフォルトは 0.96 です。

## Hyperparameters
You can further customize the model using the ObjectDetectorOptions class, which has three parameters for `SupportedModels`, `ModelOptions`, and `HParams`.

Use the `SupportedModels` enum class to specify the model architecture to use for training. The following model architectures are supported:
* MOBILENET_V2
* MOBILENET_V2_I320
* MOBILENET_MULTI_AVG
* MOBILENET_MULTI_AVG_I384

Use the `HParams` class to customize other parameters related to training and saving the model:
* `learning_rate`: Learning rate to use for gradient descent training. Defaults to 0.3.
* `batch_size`: Batch size for training. Defaults to 8.
* `epochs`: Number of training iterations over the dataset. Defaults to 30.
* `cosine_decay_epochs`: The number of epochs for cosine decay learning rate. See [tf.keras.optimizers.schedules.CosineDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/CosineDecay) for more info. Defaults to None, which is equivalent to setting it to `epochs`.
* `cosine_decay_alpha`: The alpha value for cosine decay learning rate. See [tf.keras.optimizers.schedules.CosineDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/CosineDecay) for more info. Defaults to 1.0, which means no cosine decay.

Use the `ModelOptions` class to customize parameters related to the model itself:
* `l2_weight_decay`: L2 regularization penalty used in [tf.keras.regularizers.L2](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2). Defaults to 3e-5.

Uset the `QATHParams` class to customize training parameters for Quantization Aware Training:
* `learning_rate`: Learning rate to use for gradient descent QAT. Defaults to 0.3.
* `batch_size`: Batch size for QAT. Defaults to 8
* `epochs`: Number of training iterations over the dataset. Defaults to 15.
* `decay_steps`: Learning rate decay steps for Exponential Decay. See [tf.keras.optimizers.schedules.ExponentialDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) for more information. Defaults to 8
* `decay_rate`: Learning rate decay rate for Exponential Decay. See [tf.keras.optimizers.schedules.ExponentialDecay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) for more information. Defaults to 0.96.

## ベンチマーク
以下は、サポートされているモデル アーキテクチャのベンチマーク結果の概要です。 これらのモデルは、このノートブックと同じ Android フィギュア データセットでトレーニングおよび評価されました。 モデルのベンチマーク結果を検討する際には、留意すべき重要な注意点がいくつかあります。
* Android フィギュア データセットは、62 個のトレーニング サンプルと 10 個の検証サンプルを含む小規模でシンプルなデータセットです。 データセットは非常に小さいため、トレーニング プロセスの差異によりメトリクスが大幅に変化する可能性があります。 このデータセットはデモ目的で提供されており、モデルのパフォーマンスを向上させるためにさらに多くのデータ サンプルを収集することをお勧めします。
* float32 モデルはデフォルトの HParams でトレーニングされ、int8 モデルの QAT ステップは `QATHParams(learning_rate=0.1,batch_size=4,epochs=30,decay_rate=1)` で実行されました。
* 独自のデータセットの場合、最良の結果を得るには、HParams と QATHParams の両方の値を調整する必要がある可能性があります。 トレーニング パラメーターの構成の詳細については、上記の [Hyperparameters](#hyperparameters) セクションを参照してください。
* すべての遅延数値は Pixel 6 でベンチマークされています。

## Benchmarking
Below is a summary of our benchmarking results for the supported model architectures. These models were trained and evaluated on the same android figurines dataset as this notebook. When considering the model benchmarking results, there are a few important caveats to keep in mind:
* The android figurines dataset is a small and simple dataset with 62 training examples and 10 validation examples. Since the dataset is quite small, metrics may vary drastically due to variances in the training process. This dataset was provided for demo purposes and it is recommended to collect more data samples for better performing models.
* The float32 models were trained with the default HParams, and the QAT step for the int8 models was run with `QATHParams(learning_rate=0.1, batch_size=4, epochs=30, decay_rate=1)`.
* For your own dataset, you will likely need to tune values for both HParams and QATHParams in order to achieve the best results. See the [Hyperparameters](#hyperparameters) section above for more information on configuring training parameters.
* All latency numbers are benchmarked on the Pixel 6.


<table>
<thead>
<col>
<col>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<colgroup span="2"></colgroup>
<tr>
<th rowspan="2">Model architecture</th>
<th rowspan="2">Input Image Size</th>
<th colspan="2" scope="colgroup">Test AP</th>
<th colspan="2" scope="colgroup">CPU Latency</th>
<th colspan="2" scope="colgroup">Model Size</th>
</tr>
<tr>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
<th>float32</th>
<th>QAT int8</th>
</tr>
</thead>
<tbody>
<tr>
<td>MobileNetV2</td>
<td>256x256</td>
<td>88.4%</td>
<td>73.5%</td>
<td>48ms</td>
<td>16ms</td>
<td>11MB</td>
<td>3.2MB</td>
</tr>
<tr>
<td>MobileNetV2 I320</td>
<td>320x320</td>
<td>89.1%</td>
<td>75.5%</td>
<td>75ms</td>
<td>33.38ms</td>
<td>10MB</td>
<td>3.3MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG</td>
<td>256x256</td>
<td>88.5%</td>
<td>70.0%</td>
<td>56ms</td>
<td>19ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>
<tr>
<td>MobileNet MultiHW AVG I384</td>
<td>384x384</td>
<td>92.7%</td>
<td>73.4%</td>
<td>238ms</td>
<td>41ms</td>
<td>13MB</td>
<td>3.6MB</td>
</tr>

</tbody>
</table>

